In [1]:
import numpy as np
import data

In [2]:
train = data.train
test = data.test

In [3]:
class MatrixFactorization():
    
    def __init__(self, train, test, k, learning_rate, epochs, verbose = False):
        """
        param R : Rating Matrix
        param k : latent parameter
        param learning_rate : alpha on weight update
        param epochs : training epochs
        param verbose : print status
        """
        
        self._R = train
        self._test = test
        self._I = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64) # indicator matrix
        self._n_user_rated = np.sum(self._I, axis = 1)
        self._n_item_rated = np.sum(self._I, axis = 0)
        self._num_users, self._num_items = train.shape
        # sigma?
        self._lambda = 0.01
        self._k = k
        self._learning_rate = learning_rate
        self._epochs = epochs
        self._verbose = verbose
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        # init latent features
        self._Y = np.random.normal(0, 0.1, size=(self._num_users, self._k))
        self._V = np.random.normal(0, 0.1, size=(self._num_items, self._k))
        self._W = np.random.normal(0, 0.1, size=(self._num_items, self._k))
        
        self._training_process = []
        for epoch in range(self._epochs):
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0 :
                        self.gradient_descent(i, j, self._R[i, j])
                        
            train_cost, test_cost = self.cost()
            self._training_process.append((epoch, train_cost, test_cost))
            
            if self._verbose == True and ((epoch + 1) % 10 == 0 ):
                print("Iteration : %d, train_cost = %.4f, test_cost = %.4f" % (epoch+1, train_cost, test_cost))
                
    def cost(self):
        """
        compute RMSE
        """
        xi, yi = self._R.nonzero() # 0 이 아닌 값의 index 반환
        test_x, test_y = self._test.nonzero()
        predicted = self.get_complete_matrix()
        cost_train = 0
        cost_test = 0
        
        for x, y in zip(xi, yi):
            cost_train += pow(self._R[x, y] - predicted[x, y], 2)
        
        for x, y in zip(test_x, test_y):
            cost_test += pow(self._test[x, y] - predicted[x, y], 2)
        
        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
    
    
    def gradient_descent(self, i, j, rating):
        """
        gradient descent function
        param i : user index
        param j : item index
        param rating : rating of (i, j)
        """
        prediction = self.get_prediction(i, j)
        error = rating - prediction
        
        # self._U[i, :] += self._learning_rate * ( error * self._V[j, :] - self._lambda_U / self._n_user_rated[i] * self._U[i, :])
        # self._V[j, :] += self._learning_rate * ( error * self._U[i, :] - self._lambda_V / self._n_item_rated[j] * self._V[j, :])
        
        self._Y[i, :] += self._learning_rate * (error * self._V[j, :] - self._lambda * self._Y[i, :])
        self._V[j, :] += self._learning_rate * (error * (self._Y[i, :] + self._I[i, :].dot(self._W)/self._n_user_rated[i]) - self._lambda * self._V[j, :])
        self._W += self._learning_rate * (error * np.outer(self._I[i, :], self._V[j, :])/self._n_user_rated[i])
        self._W[j, :] -= self._learning_rate * self._lambda * self._W[j, :]
        
    
    def get_prediction(self, i, j):
        """
        get predicted rating by user i on item j
        param i : user index
        param j : item index
        """
        
        return np.dot((self._Y[i, :] + self._I[i, :].dot(self._W)/self._n_user_rated[i]), self._V[j, :])
    
    
    def get_complete_matrix(self):
        """
        compute complete matrix
        """
        
        return np.dot((self._Y + self._I.dot(self._W) / self._n_user_rated[:, np.newaxis]), self._V.T)
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][2])

In [4]:
np.random.seed(7)
    
np.seterr(all="warn")
    
factorizer = MatrixFactorization(train, test, k=40, learning_rate=0.001, epochs=100, verbose=True)
# regression parameter 2개
factorizer.fit()
factorizer.print_results()

Iteration : 10, train_cost = 1.3066, test_cost = 1.4151
Iteration : 20, train_cost = 0.9889, test_cost = 1.0658
Iteration : 30, train_cost = 0.9292, test_cost = 1.0107
Iteration : 40, train_cost = 0.8987, test_cost = 0.9898
Iteration : 50, train_cost = 0.8745, test_cost = 0.9782
Iteration : 60, train_cost = 0.8510, test_cost = 0.9702
Iteration : 70, train_cost = 0.8268, test_cost = 0.9640
Iteration : 80, train_cost = 0.8019, test_cost = 0.9595
Iteration : 90, train_cost = 0.7768, test_cost = 0.9567
Iteration : 100, train_cost = 0.7517, test_cost = 0.9555
Final R matrix:
[[3.5660669  3.34076066 3.32368715 ... 0.50739024 1.12679457 1.41423354]
 [3.95305379 3.16214768 2.95584289 ... 0.7108749  1.00532895 1.21644009]
 [3.26465192 2.63730949 2.53635481 ... 0.59459288 0.70771776 1.02030751]
 ...
 [4.39816153 3.47385561 3.1971421  ... 0.80690573 0.98100972 1.29755148]
 [4.54411234 3.72469593 3.18374936 ... 0.67751735 1.09606217 1.14167587]
 [3.48622449 3.64545948 2.88218844 ... 0.67017191 0.8

## 궁금한 점
initialize latent matrix  
regularization parameter => 실제 구현된 코드들을 보면 상수로 구현이 되어 있는데 U, V matrix가 변화하면서 variance가 변화하는 것이 아닌가?  
GPU 사용하려면 어떻게 해야합니까 (속도 향상 관련)